https://towardsdatascience.com/word2vec-with-pytorch-implementing-original-paper-2cd7040120b0

https://towardsdatascience.com/running-jupyter-notebook-on-the-cloud-in-15-mins-azure-79b7797e4ef6

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.datasets import WikiText2
import pandas as pd
from nltk.corpus import brown
import numpy as np
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
import sys
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import os
import random

In [14]:
from datasets import load_dataset
wikitext2 = load_dataset("wikitext", "wikitext-2-v1")


KeyboardInterrupt: 

In [ ]:
DATA_SPLIT = "test"
text = wikitext2[DATA_SPLIT]['text']
text = [item.lower().strip() for item in text if len(item) > 0]
len(text)

2891

In [ ]:
text = [item.split(" ") + ["\n"] for item in text if "=" not in item]

In [ ]:
# Prepare Corpus
DATA_LIMIT = None #paragraph limit
all_words = []
for paragraph in text[:DATA_LIMIT]:
    all_words += paragraph
all_words = pd.Series(all_words)
len(all_words)

237655

In [ ]:
len(all_words.unique())

12476

In [ ]:
#filter out rare words
N_times = 100
v_counts = all_words.value_counts()
corpus = pd.Series([key for key in v_counts.keys() if v_counts[key] >= N_times])
corpus

0         the
1       <unk>
2           ,
3           .
4          of
        ...  
214    metres
215      near
216      east
217      navy
218    states
Length: 219, dtype: object

In [ ]:
vocab = {}
for i in range(len(corpus)):
    vocab[corpus[i]] = i
len(vocab)

219

In [ ]:
version = "april28_WT2_nodatalim_20epoch_64dim_100minf_2window"
# torch.save(vocab, f"saves/vocab_{version}.pt")

In [ ]:
enc_text = [[vocab[word] if word in vocab else vocab["<unk>"] for word in paragraph] for paragraph in text]
len(enc_text)

2176

In [ ]:
SCANNING_WINDOW = 4
#maybe need to split into paragraphs b/c different topics...
#returns context, middle word
def get_data(index, window, data):
    return list(data[index-window:index])+list(data[index+1:index+window+1]), data[index]

In [ ]:
x = []
y = []
count = 0
for paragraph in enc_text:
    if count % (len(enc_text)//16) == 0:
        print("•", end="")
    for i in range(SCANNING_WINDOW, len(paragraph) - SCANNING_WINDOW):
        if paragraph[i] == vocab["<unk>"]:
            continue
        a, b = get_data(i, SCANNING_WINDOW, paragraph)
        x.append(a)
        y.append(b)
    count+=1
    
# torch.save(x, f"train_data/{DATA_SPLIT}_data_x_wt2_window2_100minf.pt")
# torch.save(y, f"train_data/{DATA_SPLIT}_data_y_wt2_window2_100minf.pt")

••••••••••••••••